In [ ]:
import json
with open('../config.json','rb') as file:
    config = json.load(file)

kafka_host = config['kafka']['host']

In [ ]:
from confluent_kafka import Producer
import json
import pandas as pd
import time

def json_serializer(data):
    return json.dumps(data)

def delivery_report(err, msg):
    if err is not None:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

if __name__ == "__main__":
    data = pd.read_csv('data/data_recruitment_selection_update.csv')
    json_data = data.to_dict(orient='records')
    
    conf = {
        'bootstrap.servers': kafka_host,
        'acks': 'all',
    }
    
    producer = Producer(conf)

    try:
        for record in json_data:
            print(record)
            producer.produce("kelompok2-data-recruitment-selection", value=json_serializer(record), callback=delivery_report)
            producer.poll(0)
            time.sleep(10)
    except KeyboardInterrupt:
        print("Producer interrupted")
    finally:
        producer.flush()